<a href="https://colab.research.google.com/github/s1c5000/kaggle/blob/main/pima-indians-diabetes/12_%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%B1%84%EC%9A%B4%EB%92%A4%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sklearn/diabetes_feature.csv')

df_insulin = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sklearn/diabetes_fill_insulin.csv')

df['Insulin'] = df_insulin['Insulin']
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,193.458387,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,57.000730,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,184.327899,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94.000000,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.000000,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


In [ ]:
X = df[['Glucose', 'BloodPressure', 'SkinThickness',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high',
       'Insulin']]
X.shape

(768, 8)

In [ ]:
y = df['Outcome']
y.shape

(768,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape

((614, 8), (614,))

In [ ]:
X_test.shape, y_test.shape

((154, 8), (154,))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
             RandomForestClassifier(random_state=42),
             GradientBoostingClassifier(random_state=42)
            ]
estimators

[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=42, splitter='best'),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False),
 GradientBoost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

param_distributions = {"max_depth": max_depth, 
                       "max_features": max_features}

results = []
for estimator in estimators:
    result = []
    if estimator.__class__.__name__ != 'DecisionTreeClassifier':
        param_distributions["n_estimators"] = np.random.randint(100, 200, 10)
        
    clf = RandomizedSearchCV(estimator, 
                       param_distributions, 
                       n_iter=100,
                       scoring="accuracy",
                       n_jobs=-1,
                       cv=5, 
                       verbose=2
                      )

    clf.fit(X_train, y_train)
    result.append(estimator.__class__.__name__)
    result.append(clf.best_estimator_)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(X_test, y_test))
    result.append(clf.cv_results_)
    results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.2min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.6min finished


In [ ]:
df = pd.DataFrame(results, 
             columns=["estimator",'best_estimator', "best_params", "train_score", "test_score", "cv_result"])
df

,estimator,best_estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...","{'max_features': 0.4762927872759933, 'max_dept...",0.754165,0.753247,"{'mean_fit_time': [0.007079362869262695, 0.006..."
1,RandomForestClassifier,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...","{'n_estimators': 183, 'max_features': 0.717215...",0.788298,0.740260,"{'mean_fit_time': [0.5078150272369385, 0.66446..."
2,GradientBoostingClassifier,"([DecisionTreeRegressor(ccp_alpha=0.0, criteri...","{'n_estimators': 127, 'max_features': 0.948952...",0.785073,0.740260,"{'mean_fit_time': [0.5646940231323242, 0.61656..."


In [ ]:
df = df.sort_values(by='test_score', ascending=False, ignore_index=True)
df

,estimator,best_estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...","{'max_features': 0.4762927872759933, 'max_dept...",0.754165,0.753247,"{'mean_fit_time': [0.007079362869262695, 0.006..."
1,RandomForestClassifier,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...","{'n_estimators': 183, 'max_features': 0.717215...",0.788298,0.740260,"{'mean_fit_time': [0.5078150272369385, 0.66446..."
2,GradientBoostingClassifier,"([DecisionTreeRegressor(ccp_alpha=0.0, criteri...","{'n_estimators': 127, 'max_features': 0.948952...",0.785073,0.740260,"{'mean_fit_time': [0.5646940231323242, 0.61656..."


In [ ]:
pd.DataFrame(df.loc[0,'cv_result']).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
24,0.370630,0.004781,0.018392,0.000209,111,0.744472,9,"{'n_estimators': 111, 'max_features': 0.744472...",0.813008,0.829268,0.756098,0.747967,0.811475,0.791563,0.032974,1
76,0.324245,0.005703,0.017375,0.000379,106,0.744472,5,"{'n_estimators': 106, 'max_features': 0.744472...",0.837398,0.821138,0.756098,0.723577,0.811475,0.789937,0.042985,2
84,0.531903,0.003997,0.026104,0.000529,159,0.744472,8,"{'n_estimators': 159, 'max_features': 0.744472...",0.813008,0.829268,0.747967,0.747967,0.811475,0.789937,0.034831,3
39,0.542028,0.016481,0.026324,0.001753,159,0.74755,13,"{'n_estimators': 159, 'max_features': 0.747550...",0.821138,0.821138,0.739837,0.764228,0.795082,0.788285,0.032034,4
54,0.372793,0.006486,0.020754,0.003868,106,0.802595,8,"{'n_estimators': 106, 'max_features': 0.802595...",0.813008,0.837398,0.772358,0.707317,0.803279,0.786672,0.044821,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.289021,0.008026,0.018045,0.000998,101,0.336233,9,"{'n_estimators': 101, 'max_features': 0.336233...",0.756098,0.821138,0.739837,0.731707,0.786885,0.767133,0.032950,96
37,0.314667,0.006880,0.019479,0.000643,111,0.336233,9,"{'n_estimators': 111, 'max_features': 0.336233...",0.756098,0.821138,0.731707,0.739837,0.786885,0.767133,0.032950,97
48,0.543242,0.008874,0.032004,0.000631,199,0.363962,14,"{'n_estimators': 199, 'max_features': 0.363961...",0.780488,0.813008,0.723577,0.739837,0.778689,0.767120,0.031807,98
43,0.439262,0.006152,0.026550,0.000474,160,0.363962,14,"{'n_estimators': 160, 'max_features': 0.363961...",0.780488,0.813008,0.731707,0.731707,0.778689,0.767120,0.031389,98


In [ ]:
best_estimator = df.loc[0,'best_estimator']
best_estimator

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=0.4762927872759933,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=42, splitter='best')

In [ ]:
best_estimator.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=0.4762927872759933,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=42, splitter='best')

In [ ]:
y_predict = best_estimator.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.7532467532467533